# Task

This notebook produces Table 1 and accompanying numbers in Section 2 of the paper. It uses data from the OECD and the Human Mortality DataBase. 

In [1]:
import numpy as np 
from matplotlib import pyplot as plt
import pandas as pd 
from linearmodels import PanelOLS as panreg

# Data

We use data from the OECD for health care costs, income per capita and population structure. 

In [2]:
df = pd.read_stata('../data_sources/oecd/oecd_history.dta',convert_categoricals=False)

We compile from the Human Mortality DataBase data on life expectancy at birth for each country. 

In [3]:
le = pd.read_stata('../data_sources/hmd/leatbirth.dta',convert_categoricals=False)

We merge both datasets

In [4]:
df = df.merge(le,left_on=['country','year'],right_on=['country','year'],how='left')


We map country names to country codes we assigned

In [5]:
map_c = {19:'dk',16:'fr',12:'de',17:'it',14:'nl',15:'sp',13:'se',9:'uk',10:'us'}
df['cshort'] = df['cid'].replace(map_c)

We select countries we want to work with. Note that Italy does not have data from OECD going far enough in past for this exercise. Therefore, we do not use Italy. 

In [6]:
df = df[df.cid.isin([10,12,13,14,15,16,19])]
countries = ['dk','fr','nl','se','de','sp','us']

We set the period of analysis from 1970 to 2007

In [7]:
start_yr = 1970
stop_yr = 2007
df = df[(df.year>=start_yr) & (df.year<=stop_yr)]

For France, data is available every 5 years. Therefore we interpolate linearly

In [8]:
for c in ['fr']:
	df.loc[df['cshort']==c,'tothlthcpoecdcap'] = df.loc[df['cshort']==c,'tothlthcpoecdcap'].interpolate()

We create the key variables for our analysis. Everything is in real national currencies. We compute growth rates within countries, so currencies have no effect. When we do averages at the end we do averages of growth rates, so currencies do not have effects either. We compute logs to take differences later and do regressions. 

In [9]:
df = df.rename({'tothlthcpoecdcap':'pm'},axis=1)
df['log_pm'] = np.log(df['pm']) 
df = df.rename({'gdp15ncucap':'y'},axis=1)
df['log_y'] = np.log(df['y'])
df = df.rename({'propop65p':'age65p'},axis=1)
df['log_age65p'] = np.log(df['age65p'])

We create linear splines by decades

In [10]:
df['year_70'] = np.where(df['year']>=1970,np.where(df['year']<=1979,df['year']-1969,1979-1969),0)
df['year_80'] = np.where(df['year']>=1980,np.where(df['year']<=1989,df['year']-1979,1989-1979),0)
df['year_90'] = np.where(df['year']>=1990,np.where(df['year']<=1999,df['year']-1989,1999-1989),0)
df['year_00'] = np.where(df['year']>=2000,np.where(df['year']<=stop_yr,df['year']-1999,stop_yr-1999),0)
splines = ['year_70','year_80','year_90','year_00']

We set the index

In [11]:
df.set_index(['cshort','year'],inplace=True)

Looking at stats. Some missings here and there for some countries, at beginning of observation period and Germany in 1991 (reunification). We skip those observations in regressions. 

In [12]:
df[['log_pm','log_y','log_age65p','year_70','year_80','year_90','year_00']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
log_pm,262.0,8.285431,1.201817,5.787308,7.442631,7.941147,9.703233,10.410534
log_y,266.0,10.822299,1.105787,9.240627,10.047000,10.337869,12.232550,12.929209
log_age65p,266.0,2.631439,0.159236,2.251292,2.525729,2.639057,2.740840,2.985682
year_70,266.0,8.815789,2.473996,1.000000,10.000000,10.000000,10.000000,10.000000
year_80,266.0,6.184211,4.396385,0.000000,0.000000,9.500000,10.000000,10.000000
year_90,266.0,3.552632,4.316604,0.000000,0.000000,0.000000,9.000000,10.000000
year_00,266.0,0.947368,2.118439,0.000000,0.000000,0.000000,0.000000,8.000000


In [13]:
df.loc[df['log_pm'].isna(),:]

country  proptotgdp  tothlth  tothlthppp  tothlthcpoecd  \
cshort year                                                                
dk     1970      Denmark         NaN      NaN         NaN            NaN   
de     1991      Germany         NaN      NaN         NaN            NaN   
nl     1970  Netherlands         NaN      NaN         NaN            NaN   
       1971  Netherlands         NaN      NaN         NaN            NaN   

             tothltcpoecdppp  tothlthcap  tothlthpppcap  pm  \
cshort year                                                   
dk     1970              NaN         NaN            NaN NaN   
de     1991              NaN         NaN            NaN NaN   
nl     1970              NaN         NaN            NaN NaN   
       1971              NaN         NaN            NaN NaN   

             tothltcpoecdpppcap  ...  propemp   cid  leatbirth  log_pm  \
cshort year                      ...                                     
dk     1970                 NaN  ...     48.9  19.0       73.3     NaN   
de     1991                 NaN  ...      NaN  12.0       75.5     NaN   
nl     1970                 NaN  ...      NaN  14.0       73.7     NaN   
       1971                 NaN  ...      NaN  14.0       73.9     NaN   

                 log_y  log_age65p  year_70  year_80  year_90  year_00  
cshort year                                                             
dk     1970  12.107184    2.493205        1        0        0        0  
de     1991  10.228964    2.701361       10       10        2        0  
nl     1970   9.850551    2.312535        1        0        0        0  
       1971   9.881058    2.322388        2        0        0        0  

[4 rows x 54 columns]

In [14]:
data = df[['log_pm','log_y','log_age65p','year_70','year_80','year_90','year_00']].dropna()

We do a linear regression with countries fixed effects. This is a log-log specification so that coefficients measure elasticities. These will be used later. 

In [15]:
results = panreg(data['log_pm'], data[['log_y','log_age65p','year_70','year_80','year_90','year_00']], entity_effects = True).fit()

In [16]:
results

Dep. Variable:,log_pm,R-squared:,0.9276
Estimator:,PanelOLS,R-squared (Between):,0.8161
No. Observations:,262,R-squared (Within):,0.9276
Date:,"Wed, Dec 01 2021",R-squared (Overall):,0.8164
Time:,08:47:18,Log-likelihood,238.88
Cov. Estimator:,Unadjusted,,
,,F-statistic:,531.69
Entities:,7,P-value,0.0000
Avg Obs:,37.429,Distribution:,"F(6,249)"
Min Obs:,36.000,,
Max Obs:,38.000,F-statistic (robust):,531.69


We compute residuals, including the fixed effect

In [17]:
df['fe'] = results.estimated_effects
df['resid'] = results.resids
df['resid_fe'] = df['fe'] + df['resid']

We keep elasticities

In [18]:
epy = results.params['log_y']
epa = results.params['log_age65p']
epy, epa

(0.9007917236030678, 0.7043116165974213)

We pivot the data to compute growth rates more easily

In [19]:
tab = pd.pivot_table(df,index='year',columns='cshort')
tab.head()

age65p                                       cid              ...  \
cshort     de    dk    fr    nl    se    sp    us    de    dk    fr  ...   
year                                                                 ...   
1970     13.0  12.1  12.9  10.1  13.5   9.5   9.8  12.0  19.0  16.0  ...   
1971     13.3  12.3  13.0  10.2  13.7   9.6   9.9  12.0  19.0  16.0  ...   
1972     13.5  12.5  13.1  10.3  14.1   9.7  10.0  12.0  19.0  16.0  ...   
1973     13.7  12.7  13.2  10.4  14.3   9.9  10.2  12.0  19.0  16.0  ...   
1974     14.0  13.0  13.3  10.5  14.6  10.0  10.3  12.0  19.0  16.0  ...   

       year_80       year_90                    
cshort      se sp us      de dk fr nl se sp us  
year                                            
1970         0  0  0       0  0  0  0  0  0  0  
1971         0  0  0       0  0  0  0  0  0  0  
1972         0  0  0       0  0  0  0  0  0  0  
1973         0  0  0       0  0  0  0  0  0  0  
1974         0  0  0       0  0  0  0  0  0  0  

[5 rows x 392 columns]

We create a decades categorical variable to check on growth rates of the residual by decade. 

In [20]:
tab['decades'] = pd.cut(tab.index,bins=[1969,1979,1989,1999,2019])

We first check total real growth rates

In [21]:
gr = tab.loc[:,['log_pm','decades']]
for c in countries:
	gr.loc[:,('log_pm',c)] = gr.loc[:,('log_pm',c)]-gr.loc[:,('log_pm',c)].shift(1)
gr.groupby(['decades']).mean()

log_pm                                                    \
cshort              de        dk        fr        nl        se        sp   
decades                                                                    
(1969, 1979]  0.062477  0.030926  0.056782  0.040381  0.051545  0.077221   
(1979, 1989]  0.019741  0.011621  0.036308  0.013965  0.011078  0.038355   
(1989, 1999]  0.026060  0.022027  0.034124  0.036587  0.007186  0.041565   
(1999, 2019]  0.014876  0.032576  0.022151  0.035701  0.034924  0.043396   

                        
cshort              us  
decades                 
(1969, 1979]  0.049176  
(1979, 1989]  0.049442  
(1989, 1999]  0.032766  
(1999, 2019]  0.039996

We then look at residual growth rates

In [22]:
gr = tab.loc[:,['resid_fe','decades']]
for c in countries:
	gr.loc[:,('resid_fe',c)] = gr.loc[:,('resid_fe',c)]-gr.loc[:,('resid_fe',c)].shift(1)
gr.groupby(['decades']).mean()

resid_fe                                                    \
cshort              de        dk        fr        nl        se        sp   
decades                                                                    
(1969, 1979] -0.000216 -0.022284 -0.000431 -0.011773  0.000855  0.020510   
(1979, 1989]  0.003066 -0.012068  0.020095 -0.008641 -0.015238  0.003016   
(1989, 1999] -0.002119 -0.004249 -0.001429 -0.002532 -0.013894 -0.004401   
(1999, 2019] -0.029177  0.005006 -0.005675  0.002037 -0.000953  0.011922   

                        
cshort              us  
decades                 
(1969, 1979] -0.006110  
(1979, 1989]  0.022180  
(1989, 1999]  0.004201  
(1999, 2019]  0.012548

We create two periods with three years close to 1970 and three years close to 2005. 

In [23]:
tab['period'] = np.nan
tab.loc[:,'period'] = np.where((tab.index>=start_yr) & (tab.index<=start_yr+2),0,tab.loc[:,'period'])
tab.loc[:,'period'] = np.where((tab.index>=stop_yr-2) & (tab.index<=stop_yr),1,tab.loc[:,'period'])


We set the residual in levels (not used)

In [24]:
tab = tab.loc[~tab.period.isna(),:]
for c in countries:
	tab.loc[:,('resid_fe',c)] = np.exp(tab.loc[:,('resid_fe',c)])

We select what we will look at

In [25]:
tab = tab.loc[:,['pm','y','age65p','resid_fe','totpop','leatbirth','period']]

We do means of both pre and post periods

In [26]:
comp = tab.groupby('period').mean()
comp.index=['1970','2005']

/Users/loulou/.local/lib/python3.8/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


We compute average growth rates between pre and post periods for various outcomes.

In [27]:
nyears = stop_yr - start_yr
for o in ['pm','y','age65p','resid_fe','leatbirth']:
	for c in countries:
		comp.loc['total',(o,c)] = (comp.loc['2005',(o,c)]/comp.loc['1970',(o,c)])**(1/nyears)-1.0

In [28]:
comp

pm                                                        \
cshort           de            dk           fr           nl            se   
1970    1224.219333  14655.579500   897.815600  1279.080000  14005.546000   
2005    3371.977667  32191.699000  3279.448333  3638.045000  32271.261667   
total      0.027762      0.021495     0.035633     0.028654      0.022817   

                                             y                               \
cshort           sp           us            de             dk            fr   
1970     381.980667  1754.458667  21257.766667  185997.833333  16524.300000   
2005    1990.356333  7912.289000  32962.200000  357608.766667  32392.566667   
total      0.045624     0.041550      0.011926       0.017825      0.018358   

        ...       totpop                          leatbirth             \
cshort  ...           se            sp        us         de         dk   
1970    ...  8087.800000  34214.366667  207536.3  70.800000  73.333333   
2005    ...  9086.066667  44425.766667  298375.9  79.766667  78.366667   
total   ...          NaN           NaN       NaN   0.003228   0.001796   

                                                               
cshort         fr         nl         se         sp         us  
1970    72.233333  73.800000  74.800000  72.000000  71.133333  
2005    80.866667  79.900000  80.933333  80.866667  77.833333  
total    0.003056   0.002149   0.002132   0.003144   0.002436  

[3 rows x 42 columns]

We setup Table 1. 

In [29]:
table = pd.DataFrame(index=['GDP growth','Pop Aging','Unexplained','Total'],columns=countries)

Total growth can be read directly

In [30]:
table.loc['Total',:] = comp.loc['total',('pm',)]

/Users/loulou/.local/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


Next we compute growth for each component of Table 1 using elasticities and growth rates of the various variables

In [31]:
table.loc['GDP growth',:] = comp.loc['total',('y',)]*epy
table.loc['Pop Aging',:] = comp.loc['total',('age65p',)]*epa
table.loc['Unexplained',:] = table.loc['Total',:] - table.loc['Pop Aging',:] - table.loc['GDP growth',:]


Here is the data for Table 1

In [32]:
table

,dk,fr,nl,se,de,sp,us
GDP growth,0.016056,0.016537,0.017302,0.016312,0.010743,0.019754,0.017838
Pop Aging,0.003957,0.00486,0.006416,0.004288,0.007105,0.010307,0.004453
Unexplained,0.001482,0.014236,0.004936,0.002216,0.009914,0.015562,0.019259
Total,0.021495,0.035633,0.028654,0.022817,0.027762,0.045624,0.04155


We add the column with European population averaged rates

In [33]:
eu_countries = [c for c in countries if c!='us']
pop_eu = comp.loc['2005',('totpop',)]
pop_eu = pop_eu[eu_countries]
pop_eu = pop_eu/pop_eu.sum()
for i in table.index:
	table.loc[i,'eu'] = np.sum(table.loc[i,eu_countries]*pop_eu)
table

/Users/loulou/.local/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


,dk,fr,nl,se,de,sp,us,eu
GDP growth,0.016056,0.016537,0.017302,0.016312,0.010743,0.019754,0.017838,0.015060
Pop Aging,0.003957,0.00486,0.006416,0.004288,0.007105,0.010307,0.004453,0.006860
Unexplained,0.001482,0.014236,0.004936,0.002216,0.009914,0.015562,0.019259,0.011397
Total,0.021495,0.035633,0.028654,0.022817,0.027762,0.045624,0.04155,0.033317


In [34]:
for i in table.columns:
	table[i] = table[i].astype('float64')

# Table 1

This is Table 1 in the paper

In [35]:
table.loc[table.index!='Total',['de','dk','fr','nl','se','sp','us','eu']].round(3).to_latex('../tables/tab_1_growth.tex')